In [ ]:
### Representation Similarity Analysis ###

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/dis.experiments.4'
import sys
sys.path.append(drive_PATH)
# drive_PATH = ''

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# import os, sys
from matplotlib import pyplot as plt
%matplotlib inline

import utils.cca_core as cca_core
def _plot_helper(arr, xlabel, ylabel):
    plt.plot(arr, lw=2.0)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()

In [ ]:
baseline_PATH = drive_PATH + "/res/activations/Baseline/test.csv"
poe_PATH = drive_PATH + "/res/activations/POE/test.csv"
dfr_PATH = drive_PATH + "/res/activations/DFR/test.csv"

baseline_act = pd.read_csv(baseline_PATH, delimiter=',', header=0, index_col=[0,1])
poe_act = pd.read_csv(poe_PATH, delimiter=',', header=0, index_col=[0,1])
dfr_act = pd.read_csv(dfr_PATH, delimiter=',', header=0, index_col=[0,1])

In [ ]:
keys = np.unique(np.array([np.array(a) for a in baseline_act.index])[:,0]) # same for dfr

baseline_act_dict = {}
poe_act_dict = {}
dfr_act_dict = {}

for key in keys:
  baseline_act_dict[key] = np.array(baseline_act.loc[key].dropna(axis=1))
  poe_act_dict[key] = np.array(poe_act.loc['nli_model.'+key].dropna(axis=1))
  dfr_act_dict[key] = np.array(dfr_act.loc[key].dropna(axis=1))
  print(baseline_act_dict[key].shape, poe_act_dict[key].shape , dfr_act_dict[key].shape)

(512, 1600) (512, 1600) (512, 1600)
(512, 1600) (512, 1600) (512, 1600)
(2, 1600) (2, 1600) (2, 1600)
(256, 3200) (256, 3200) (256, 3200)


In [ ]:
keys

array(['classifier.classifier.1', 'classifier.classifier.4',
       'classifier.classifier.7', 'encoder'], dtype='<U23')

In [ ]:
# two models to compare
model1_act = dfr_act_dict
model2_act = poe_act_dict

# layer to compare
layer_name = 'encoder'
print(layer_name)

# set activations
acts1 = model1_act[layer_name]
acts2 = model2_act[layer_name]

encoder


In [ ]:
import utils.cca_core as cca_core
results = cca_core.get_cca_similarity(acts1, acts2, verbose=True)

adding eps to diagonal and taking inverse
taking square root
dot products...
trying to take final svd


LinAlgError: ignored

In [ ]:
print("Returned Information:")
print(results.keys())

Returned Information:


NameError: ignored

In [ ]:
results = cca_core.get_cca_similarity(acts1, acts2, epsilon=1e-10, verbose=False)

LinAlgError: ignored

In [ ]:
_plot_helper(results["cca_coef1"], "CCA Coef idx", "CCA Coef value")
print("Mean CCA similarity", np.mean(results["cca_coef1"]))

In [ ]:

_, s, v = np.linalg.svd(acts1 - np.mean(acts1, axis=1, keepdims=True), full_matrices=False)
_plot_helper(s, "singular value idx", "singular value")
plt.plot([20]*100, np.linspace(0, 2700, 100), linestyle="--")
print("Fraction of variance explained by 20 singular vectors", np.sum(s[:20])/np.sum(s))

In [ ]:
# creating a random baseline
b1 = np.random.randn(*acts1.shape)
b2 = np.random.randn(*acts2.shape)

In [ ]:
baseline = cca_core.get_cca_similarity(b1, b2, epsilon=1e-10, verbose=False)
_plot_helper(baseline["cca_coef1"], "CCA coef idx", "CCA coef value")
print("Baseline Mean CCA similarity", np.mean(baseline["cca_coef1"]))
print("MNLI Mean CCA similarity", np.mean(results["cca_coef1"]))

In [ ]:
### 1. Averaging Top CCA Values
# comparing different results
print("Results when averaging top CCA values")
print("Baseline", np.mean(baseline["cca_coef1"][:20]), "and MNLI", np.mean(results["cca_coef1"][:20]))

In [ ]:
# 2. Using SVCCA
print("Results using SVCCA keeping 20 dims")
# Mean subtract activations
cacts1 = acts1 - np.mean(acts1, axis=1, keepdims=True)
cacts2 = acts2 - np.mean(acts2, axis=1, keepdims=True)

# Perform SVD
U1, s1, V1 = np.linalg.svd(cacts1, full_matrices=False)
U2, s2, V2 = np.linalg.svd(cacts2, full_matrices=False)

svacts1 = np.dot(s1[:20]*np.eye(20), V1[:20])
# can also compute as svacts1 = np.dot(U1.T[:20], cacts1)
svacts2 = np.dot(s2[:20]*np.eye(20), V2[:20])
# can also compute as svacts1 = np.dot(U2.T[:20], cacts2)

svcca_results = cca_core.get_cca_similarity(svacts1, svacts2, epsilon=1e-10, verbose=False)

# Mean subtract baseline activations
cb1 = b1 - np.mean(b1, axis=0, keepdims=True)
cb2 = b2 - np.mean(b2, axis=0, keepdims=True)

# Perform SVD
Ub1, sb1, Vb1 = np.linalg.svd(cb1, full_matrices=False)
Ub2, sb2, Vb2 = np.linalg.svd(cb2, full_matrices=False)

svb1 = np.dot(sb1[:20]*np.eye(20), Vb1[:20])
svb2 = np.dot(sb2[:20]*np.eye(20), Vb2[:20])

svcca_baseline = cca_core.get_cca_similarity(svb1, svb2, epsilon=1e-10, verbose=False)
print("Baseline", np.mean(svcca_baseline["cca_coef1"]), "and MNLI", np.mean(svcca_results["cca_coef1"]))

plt.plot(svcca_baseline["cca_coef1"], lw=2.0, label="baseline")
plt.plot(svcca_results["cca_coef1"], lw=2.0, label="MNLI")
plt.xlabel("Sorted CCA Correlation Coeff Idx")
plt.ylabel("CCA Correlation Coefficient Value")
plt.legend(loc="best")
plt.grid()

In [ ]:
# 3. Using PWCCA
import utils.pwcca as pwcca
pwcca_mean, w, _ = pwcca.compute_pwcca(acts1, acts2, epsilon=1e-10)
pwcca_baseline, wb, _ = pwcca.compute_pwcca(b1, b2, epsilon=1e-10)
print("Results using PWCCA")
print("Baseline", pwcca_baseline, "and MNLI", pwcca_mean)